In [1]:
import pandas as pd

In [2]:
from glob import glob

In [3]:
print("Bem Vindo(a) :D ")

Bem Vindo(a) :D 


In [4]:
print("Lendo bases...")

Lendo bases...


In [5]:
df_base_atestados = pd.read_excel(glob("bases/*")[0], index_col = None)

In [6]:
df_base_atestados.head(1)

,aaDATA no SAP,MAT,NOME,IDADE,SEXO,FUNÇÃO,LOJA,DATA INICIAL ATESTADO,DATA FINAL ATESTADO,VAI PRO INSS?,DIAS PERDIDOS,CID-10,GRUPO DE DOENÇAS,LÇDO NO SOC?,"OBS.: > DE 15 DIAS,SÓ CONTABILIZAR OS 15 DIAS QUE A EMPESA PAGA, POIS RESTANTE É PELO INSS",Unnamed: 15,Unnamed: 16
0,2019-05-10 00:00:00,4402,MARIA LUIZA BARBOSA DE SOUZA RICARDO,55,F,VENDEDOR VITRINE,VENDAS - BARROSO,2019-05-08 00:00:00,2019-05-08 00:00:00,NÃO,1,Z019,EXAME,SIM 30.05.2019,NaN,NaN,NaN


In [7]:
df_base_atestados.columns = ['data_sap','matricula','nome','idade','sexo','funcao','loja','data_inicial','data_final','vai_inss','dias_perdidos','cib_10','grupo_doenca','lcdo_soc','','','']

In [8]:
df_base_atestados_filtrado = df_base_atestados[['data_sap','matricula','nome','idade','sexo','funcao','loja','data_inicial','data_final','vai_inss','dias_perdidos','cib_10','grupo_doenca','lcdo_soc']].copy()

In [9]:
df_base_atestados_filtrado = df_base_atestados_filtrado[~df_base_atestados_filtrado['matricula'].isnull()].copy()

In [10]:
df_base_atestados_filtxsdsrado = df_base_atestados_filtrado[df_base_atestados_filtrado['nome'] != 'NOME'].copy()

In [11]:
df_atestados_meses = df_base_atestados_filtrado[['data_inicial','dias_perdidos']].copy()

In [12]:
### Pegando somente o dado que necessita que é o mês
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].astype(str)

In [13]:
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(".","-")
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(":","-")
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(" ","-")

In [14]:
df_atestados_meses['mes'] = df_atestados_meses['data_inicial'].apply(lambda item: str(item).split('-')[1])

In [15]:
df_atestados_meses.drop(columns = {'data_inicial'}, inplace = True)

In [16]:
df_atestados_meses = df_atestados_meses[~df_atestados_meses['mes'].str.contains('INICIAL')]

In [17]:
df_atestados_meses['mes'] = df_atestados_meses['mes'].astype(int)

In [18]:
df_atestados_meses['mes_copiado'] = df_atestados_meses['mes']  

In [19]:
## quantidade de dias perdidos por mês e quantidade de atestados por mês
df_atestados_meses = df_atestados_meses.groupby('mes', as_index = False).agg({'mes_copiado': 'count','dias_perdidos':'sum'})

In [20]:
df_atestados_meses.rename(columns = {'mes_copiado':'QTE DE LICENÇAS MÉDICAS','dias_perdidos':'QTE DIAS PERDIDOS'}, inplace = True)

In [21]:
df_atestados_meses

,mes,QTE DE LICENÇAS MÉDICAS,QTE DIAS PERDIDOS
0,5,152,360
1,6,162,341
2,7,200,485
3,8,199,426
4,9,153,332
5,10,199,411


In [22]:
nome_mes = {1: 'JAN', 2: 'FEV', 3:'MARC', 4:'ABR', 5:'MAIO', 6:'JUN', 7:'JUL', 8:'AGO', 9:'SET', 10:'OUT', 11:'NOV' , 12: 'DEZ'}

In [23]:
df_atestados_meses['nome_mes'] = df_atestados_meses['mes'].map(nome_mes)

In [24]:
df_atestados_meses

## fazer um sort na coluna mes em ordem crescente ## falta fazerrrrr

,mes,QTE DE LICENÇAS MÉDICAS,QTE DIAS PERDIDOS,nome_mes
0,5,152,360,MAIO
1,6,162,341,JUN
2,7,200,485,JUL
3,8,199,426,AGO
4,9,153,332,SET
5,10,199,411,OUT


In [25]:
df_antes_trans = df_atestados_meses.T

In [26]:
df_antes_trans

,0,1,2,3,4,5
mes,5,6,7,8,9,10
QTE DE LICENÇAS MÉDICAS,152,162,200,199,153,199
QTE DIAS PERDIDOS,360,341,485,426,332,411
nome_mes,MAIO,JUN,JUL,AGO,SET,OUT


In [27]:
df_antes_trans.columns = df_antes_trans.loc['nome_mes'] 
df_antes_trans.columns.name = None

In [28]:
df_antes_trans.drop(['mes', 'nome_mes'])

,MAIO,JUN,JUL,AGO,SET,OUT
QTE DE LICENÇAS MÉDICAS,152,162,200,199,153,199
QTE DIAS PERDIDOS,360,341,485,426,332,411


## ------ Reorganizando os meses com os respectivos nomes -----